In [37]:
%pip install pandas numpy regex textblob transformers torch scikit-learn


  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 25.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 29.5 MB/s eta 0:00:00a 0:00:01
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
import pandas as pd
import numpy as np
import regex as re
import string

In [8]:
train_csv = pd.read_csv('Data/train.csv')

In [10]:
train_csv.head()

,tweet_id,user_handle,timestamp,tweet_text,candidate,party,retweets,likes,sentiment
0,1,@user123,2024-11-03 08:45:00,Excited to see Kamala Harris leading the Democ...,Kamala Harris,Democratic Party,120,450,positive
1,2,@politicsFan,2024-11-03 09:15:23,Donald Trump's policies are the best for our e...,Donald Trump,Republican Party,85,300,positive
2,3,@greenAdvocate,2024-11-03 10:05:45,Jill Stein's environmental plans are exactly w...,Jill Stein,Green Party,60,200,positive
3,4,@indieVoice,2024-11-03 11:20:10,Robert Kennedy offers a fresh perspective outs...,Robert Kennedy,Independent,40,150,neutral
4,5,@libertyLover,2024-11-03 12:35:55,Chase Oliver's libertarian stance promotes tru...,Chase Oliver,Libertarian Party,30,120,positive


Data Preprocessing

In [16]:
# Remove HMTL Tages, lowercase, remove @mentions
clean = re.compile('<.*?>')
train_csv['tweet_text'] = train_csv['tweet_text'].str.lower()
train_csv['tweet_text'] = train_csv['tweet_text'].apply(lambda x: re.sub(clean, '',x) )
train_csv['tweet_text'] = train_csv['tweet_text'].str.replace(r'@\S+', '', regex=True)


In [21]:
#remove Puncuations
train_csv['tweet_text'] = train_csv['tweet_text'].apply(lambda x : x.translate(str.maketrans('', '',string.punctuation)))

In [32]:
# spelling corrections
from textblob import TextBlob
train_csv['tweet_text'] = train_csv['tweet_text'].apply(lambda x: TextBlob(x).correct().string)

In [38]:
import random
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

In [40]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [43]:
text = train_csv['tweet_text'][0]

encoding = tokenizer.batch_encode_plus( [text],# List of input texts
    padding=True,              # Pad to the maximum sequence length
    truncation=True,           # Truncate to the maximum sequence length if necessary
    return_tensors='pt',      # Return PyTorch tensors
    add_special_tokens=True    # Add special tokens CLS and SEP
)

input_ids = encoding['input_ids']  # Token IDs
# print input IDs
print(f'Input ID: {input_ids}')
attention_mask = encoding['attention_mask']  # Attention mask
# print attention mask
print(f'Attention mask: {attention_mask}')

Input ID: tensor([[  101,  7568,  2000,  2156, 28935,  5671,  2877,  1996,  3537,  3715,
           102]])
Attention mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [44]:
# Generate embeddings using BERT model
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    word_embeddings = outputs.last_hidden_state  # This contains the embeddings

# Output the shape of word embeddings
print(f'Shape of Word Embeddings: {word_embeddings.shape}')

Shape of Word Embeddings: torch.Size([1, 11, 768])


In [46]:
# Decode the token IDs back to text
decoded_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
#print decoded text
print(f'Decoded Text: {decoded_text}')
# Tokenize the text again for reference
tokenized_text = tokenizer.tokenize(decoded_text)
#print tokenized text
print(f'tokenized Text: {tokenized_text}')
# Encode the text
encoded_text = tokenizer.encode(text, return_tensors='pt')  # Returns a tensor
# Print encoded text
print(f'Encoded Text: {encoded_text}')

Decoded Text: excited to see mala harris leading the democratic charge
tokenized Text: ['excited', 'to', 'see', 'mala', 'harris', 'leading', 'the', 'democratic', 'charge']
Encoded Text: tensor([[  101,  7568,  2000,  2156, 28935,  5671,  2877,  1996,  3537,  3715,
           102]])


In [50]:
word_embeddings

tensor([[[-0.1750,  0.0534, -0.1162,  ..., -0.0963,  0.2406,  0.3589],
         [ 0.6159, -0.6472, -0.3491,  ..., -0.1919,  0.4275,  0.5265],
         [ 0.2682, -0.4770,  0.1700,  ..., -0.3569,  0.2511,  0.0313],
         ...,
         [-0.4254, -0.6637, -0.1005,  ..., -0.3504, -0.3146,  0.5374],
         [-0.5015, -0.9129, -0.1265,  ...,  0.1249,  0.0892, -0.3274],
         [ 0.6415,  0.0750, -0.3870,  ...,  0.1416, -0.6869, -0.0764]]])

In [51]:
def word_embeddings_BERT(text):
    text = train_csv['tweet_text'][0]

    encoding = tokenizer.batch_encode_plus( [text],# List of input texts
        padding=True,              # Pad to the maximum sequence length
        truncation=True,           # Truncate to the maximum sequence length if necessary
        return_tensors='pt',      # Return PyTorch tensors
        add_special_tokens=True    # Add special tokens CLS and SEP
    )

    input_ids = encoding['input_ids']  # Token IDs
    # print input IDs
    # print(f'Input ID: {input_ids}')
    attention_mask = encoding['attention_mask']  # Attention mask
    # print attention mask
    # print(f'Attention mask: {attention_mask}')
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        word_embeddings = outputs.last_hidden_state  # This contains the embeddings

    return word_embeddings

In [61]:
word_embeddings_BERT(train_csv['tweet_text'][0]).shape

torch.Size([1, 11, 768])

In [62]:
train_csv['tweet_text_embeded'] = train_csv['tweet_text'].apply(lambda x: word_embeddings_BERT(x)[0])

In [64]:
train_csv.to_csv('Data/train_embeded.csv', index=False)